<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
<a href="https://sebastianraschka.com">Sebastian Raschka</a>의 책 <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a>를 위한 보충 코드<br>
<br>코드 저장소: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# OpenAI API를 사용한 지시(Instruction) 응답 평가

- 이 노트북은 OpenAI의 GPT-4 API를 사용하여 지시 튜닝(instruction finetuned)된 LLM의 응답을 평가합니다. 평가는 생성된 모델 응답이 포함된 JSON 형식의 데이터셋을 기반으로 합니다. 예시는 다음과 같습니다:



```python
{
    "instruction": "What is the atomic number of helium?",
    "input": "",
    "output": "The atomic number of helium is 2.",               # <-- 테스트 세트에 정답(Target)으로 주어진 값
    "model 1 response": "\nThe atomic number of helium is 2.0.", # <-- LLM 모델 1의 응답
    "model 2 response": "\nThe atomic number of helium is 3."    # <-- 2번째 LLM 모델의 응답
},
```

In [ ]:
%pip install python-dotenv openai

In [ ]:
from importlib.metadata import version

pkgs = ["openai",  # OpenAI API
        "tqdm",    # 진행률 표시줄(Progress bar)
        "python-dotenv"  # .env 파일에서 환경 변수 로드
        ]

for p in pkgs:
    print(f"{p} version: {version(p)}")

openai version: 1.30.3
tqdm version: 4.66.2


## OpenAI API 테스트

- 먼저, OpenAI API가 올바르게 설정되었는지 테스트해보겠습니다.
- 아직 계정이 없다면 https://platform.openai.com/ 에서 계정을 생성해야 합니다.
- GPT-4 API는 무료가 아니므로 계정에 자금을 충전해야 합니다 (https://platform.openai.com/settings/organization/billing/overview 참조).
- 이 노트북의 코드를 사용하여 실험을 실행하고 약 200개의 평가를 생성하는 데 드는 비용은 작성 시점 기준으로 약 $0.26(26센트)입니다.

- 먼저 OpenAI API 비밀 키(secret key)를 제공해야 합니다. 키는 https://platform.openai.com/api-keys 에서 찾을 수 있습니다.
- 이 키를 다른 사람과 공유하지 않도록 주의하세요.
- 비밀 키(`"sk-..."`)를 이 폴더에 있는 `config.json` 파일에 추가하세요.

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# 1. .env 파일에 정의된 환경 변수를 시스템으로 로드합니다.
load_dotenv()

# 2. os.getenv를 사용해 환경 변수 값을 가져옵니다.
api_key = os.getenv("OPENAI_API_KEY")

# 3. 클라이언트를 생성합니다.
# (참고: OpenAI 라이브러리는 환경 변수 'OPENAI_API_KEY'가 로드되어 있으면 
#  api_key 인자를 생략해도 자동으로 감지합니다.)
client = OpenAI(api_key=api_key)

print("OpenAI 클라이언트가 성공적으로 설정되었습니다.")

OpenAI 클라이언트가 성공적으로 설정되었습니다.


- 먼저, 의도한 대로 작동하는지 확인하기 위해 간단한 예제로 API를 테스트해 보겠습니다:

In [3]:
def run_chatgpt(prompt, client, model="gpt-5-mini"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content


prompt = "Respond with 'hello world' if you got this message."
run_chatgpt(prompt, client)

'hello world'

## JSON 항목 로드

- 여기서는 테스트 데이터셋과 모델 응답을 JSON 파일로 저장했다고 가정하고, 이를 다음과 같이 로드합니다:

In [12]:
import json
json_file = "outputs/instruction-data-with-response.json"

with open(json_file, "r") as file:
    json_data = json.load(file)

print("Number of entries:", len(json_data))

Number of entries: 110


- 이 파일의 구조는 다음과 같으며, 테스트 데이터셋의 정답(`'output'`)과 두 가지 다른 모델의 응답(`'model 1 response'` 및 `'model 2 response'`)이 포함되어 있습니다:

In [13]:
json_data[0]

{'instruction': 'Rewrite the sentence using a simile.',
 'input': 'The car is very fast.',
 'output': 'The car is as fast as lightning.',
 'model_response': 'The car is as fast as a bullet.'}

- 아래는 시각화를 위해 입력을 포맷팅하는 작은 유틸리티 함수입니다:

In [16]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text

- 이제 OpenAI API를 사용하여 모델 응답을 비교해 보겠습니다(시각적 비교를 위해 처음 5개 응답만 평가합니다):

In [18]:
for entry in json_data[:3]:
    prompt = (
        f"Given the input `{format_input(entry)}` "
        f"and correct output `{entry['output']}`, "
        f"score the model response `{entry['model_response']}`"
        f" on a scale from 0 to 100, where 100 is the best score. "
    )
    print("\nDataset response:")
    print(">>", entry['output'])
    print("\nModel response:")
    print(">>", entry["model_response"])
    print("\nScore:")
    print(">>", run_chatgpt(prompt, client))
    print("\n-------------------------")


Dataset response:
>> The car is as fast as lightning.

Model response:
>> The car is as fast as a bullet.

Score:
>> 100

Explanation: The response correctly rewrites the sentence as a simile ("as fast as a bullet"), is grammatical, and preserves the intended meaning (extreme speed). It's an acceptable alternative to "as fast as lightning."

-------------------------

Dataset response:
>> The type of cloud typically associated with thunderstorms is cumulonimbus.

Model response:
>> The type of cloud associated with thunderstorms is a cumulus cloud.

Score:
>> 20

Incorrect: thunderstorms are typically produced by cumulonimbus clouds. The answer "cumulus" is not specific (only cumulonimbus—an intense, vertically developed cumulus type—is the correct thunderstorm cloud), though cumulus congestus can develop into cumulonimbus, so minor partial credit.

-------------------------

Dataset response:
>> Jane Austen.

Model response:
>> The author of 'Pride and Prejudice' is Jane Austen.

Sco

- 응답이 매우 장황하다는 점을 유의하세요. 어떤 모델이 더 나은지 정량화하기 위해 점수만 반환받도록 하겠습니다:

In [ ]:
def generate_model_scores(json_data, json_key, client=client):
    scores = []
    for entry in tqdm(json_data, desc="Scoring entries"):
        prompt = (
            f"Given the input `{format_input(entry)}` "
            f"and correct output `{entry['output']}`, "
            f"score the model response `{entry[json_key]}`"
            f" on a scale from 0 to 100, where 100 is the best score. "
            f"Respond with the integer number only."
        )
        score = run_chatgpt(prompt, client)
        try:
            scores.append(int(score))
        except ValueError:
            print(f"Could not convert score: {score}")
            continue

    return scores


scores = generate_model_scores(json_data, "model_response", client=client)
print(f"Number of scores: {len(scores)} of {len(json_data)}")
print(f"Average score: {sum(scores)/len(scores):.2f}\n")

Scoring entries:  28%|██▊       | 31/110 [02:59<07:05,  5.39s/it]

- 참고로 OpenAI의 GPT 모델은 랜덤 시드를 설정하더라도 비결정적(non-deterministic)이므로 응답 점수가 달라질 수 있습니다.